# 16.DDPG（deep deterministic policy gradient）：深度确定性策略梯度
1. **在线策略算法** 的 **样本效率（sample efficiency）** 都比较低。 **DQN 算法** 虽然为 **离线策略学习**，但是它只能处理 **动作空间有限** 的环境，即使将动作空间离散化，但这会比较粗糙，无法精细控制。
2. ***深度确定性策略梯度（deep deterministic policy gradient，DDPG）*** 结合了 **确定性策略梯度（Deterministic Policy Gradient, DPG）** 与 **DQN 算法** 的思想，通过构造一个 **确定性策略**，来处理 **动作空间无限** 的环境并且是 **离线策略算法**。
3. **DDPG** 也属于一种 **Actor-Critic** 算法。不过之前学习的 REINFORCE、TRPO 和 PPO 学习 **随机性策略**，而 **DDPG** 则学习一个 **确定性策略**。
> **DDPG** 最早由 **Google DeepMind** 于 2015 年在 [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971#) 中提出，主要用于解决传统 DQN **不能处理连续动作空间** 的问题

## 16.1 DDPG 算法原理

### 确定性策略梯度定理
> 传统**策略梯度方法（PG）** ：
$$J(\theta)=\mathbb{E}_{s\sim\nu^\pi,a\sim\pi_\theta}
\begin{bmatrix}
R
\end{bmatrix}$$
> 策略是 **随机性** 的，计算需要依赖于 **概率性动作分布：$a\sim\pi_{\theta}(\cdot|s)$**，这在面对 **连续动作空间** 时变得非常复杂
> **确定性策略梯度定理（Deterministic Policy Gradient Theorem, DPG 定理）** 假设有一个 **确定性策略 $\mu$**，**直接输出动作：$a=\mu_\theta(s)$**，而不是动作的概率分布：$a\sim\pi_{\theta}(\cdot|s)$
> 此时：
$$J(\mu_\theta)=\int_{\mathcal{S}}\nu^{\mu_\theta}(s)r(s,\mu_\theta(s))\mathrm{d}s=\mathbb{E}_{s\sim\nu^{\mu_\theta}}[r(s,\mu_\theta(s))]$$
> 可以避免计算**概率分布**的导数，简化梯度计算过程
> **离线策略形式**：
$$J(\theta)=\int_{\mathcal{S}}\nu^{\pi_\beta} V^{\mu_\theta}(s)\mathrm{d}s=\int_{\mathcal{S}}\nu^{\pi_\beta} Q^{\mu_\theta}(s,\mu_\theta(s))\mathrm{d}s=\mathbb{E}_{s\sim\nu^{\pi_\beta}}
\begin{bmatrix}
Q^{\mu_\theta}(s,a)|_{a=\mu_\theta(s)}
\end{bmatrix}$$
- $\pi_{\beta}$是用来收集数据的行为策略
> 其 **梯度** 为（证明过程与 **策略梯度定理** 相似）：
$$\nabla_\theta J(\pi_\theta)=\mathbb{E}_{s\sim\nu^{\pi_\beta}}
\begin{bmatrix}
\nabla_\theta\mu_\theta(s)\nabla_aQ_\omega^{\mu_\theta}(s,a)|_{a=\mu_\theta(s)}
\end{bmatrix}$$
- 可见，$Q_\omega$ 就是**Critic**，$\mu_\theta$ 就是**Actor**，这是一个 **Actor-Critic** 的框架

### DDPG 中 Actor 的目标网络

Double DQN、随机噪声

### DDPG 算法伪代码

#### 初始化
- 初始化 Actor 网络和 Critic 网络
- 初始化目标网络 (Actor_target, Critic_target)
- 初始化经验回放缓冲区

#### 每个训练回合
对于每一回合：

1. 初始化状态 `s`

2. **每个时间步 `t`**：
   - 使用 Actor 网络选择动作 `a`，并加上探索噪声：`a = actor(s) + 噪声`
   - 执行动作 `a`，观察下一个状态 `s'` 和奖励 `r`
   - 将经验 `(s, a, r, s')` 存储到经验回放缓冲区

3. **从经验回放中采样一批数据**：
   - 从回放缓冲区中随机采样：`(s_batch, a_batch, r_batch, s'_batch)`

4. **更新 Critic 网络**：
   - 计算目标 Q 值：`y = r_batch + gamma * critic_target(s'_batch, actor_target(s'_batch))`
   - 最小化误差：`Q(s, a) - y`，更新 Critic 网络

5. **更新 Actor 网络**：
   - 计算策略梯度：`∇_a Q(s, a)` 的梯度
   - 使用策略梯度更新 Actor 网络参数

6. **软更新目标网络**：
   - 使用软更新公式：`目标网络参数 = tau * 当前网络参数 + (1 - tau) * 目标网络参数`

#### 结束


## 16.2 DDPG 代码实践（Pendulum-v1）